## Projeto Final - Segunda Entrega

In [1]:
# Instalação Requirements
!pip install -r requirements.txt

^C


In [2]:
# Importando as bibliotecas para o projeto
import pandas as pd
import requests 
from datetime import datetime
from plyer import notification

In [3]:
# Função para exibir as notificações de erro no carregamento da API
def alerta(api_nome, status_code):
    notification.notify(
        title="Erro na API",
        message=f"Falha no carregamento da API {api_nome}\nErro: {status_code}\n{datetime.now()}",
        app_name="alerta",
        timeout=10
    )

In [5]:
# Criando dicionário com as URLs das APIs
apis = {
    "temperaturas": "https://global-warming.org/api/temperature-api",
    "gelo": "https://global-warming.org/api/arctic-api",
    "oceanos": "https://global-warming.org/api/ocean-warming-api"
}

def carregar_api(url, nome_api):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
                
        # Transformar os dados em um dataframe
        try:
            df = pd.DataFrame(data)
            print(f"Primeiras linhas do DataFrame para a API {nome_api}:\n{df.head()}")
            return df
        except ValueError:
            alerta(nome_api, "Dados retornados não podem ser convertidos em um dataframe")
            return None
    else:
        alerta(nome_api, f"Erro ao acessar API. Código HTTP: {response.status_code}")
        return None

#Carregando os dados
for nome_api, url in apis.items():
    if nome_api == "temperaturas":
        data_temp = carregar_api(url, nome_api)
    elif nome_api == "gelo":
        data_ice = carregar_api(url, nome_api)
    elif nome_api == "oceanos":
        data_ocean = carregar_api(url, nome_api)


Primeiras linhas do DataFrame para a API temperaturas:
  error                                             result
0  None  {'time': '1880.04', 'station': '-0.29', 'land'...
1  None  {'time': '1880.13', 'station': '-0.48', 'land'...
2  None  {'time': '1880.21', 'station': '-0.32', 'land'...
3  None  {'time': '1880.29', 'station': '-0.52', 'land'...
4  None  {'time': '1880.38', 'station': '-0.22', 'land'...
Primeiras linhas do DataFrame para a API gelo:
            error                                         arcticData
description  None  {'title': 'Global Sea Ice Extent (1979-2024)',...
data         None  {'197901': {'value': 20.81, 'anom': 1.79, 'mon...
Primeiras linhas do DataFrame para a API oceanos:
     error result
1851  None  -0.05
1852  None   0.01
1853  None   0.05
1854  None   0.00
1855  None  -0.01


In [6]:
# checar o type dos itens
data_temp.info()
data_ice.info()
data_ocean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1739 entries, 0 to 1738
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   error   0 non-null      object
 1   result  1739 non-null   object
dtypes: object(2)
memory usage: 27.3+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, description to data
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   error       0 non-null      object
 1   arcticData  2 non-null      object
dtypes: object(2)
memory usage: 48.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Index: 174 entries, 1851 to 2024
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   error   0 non-null      object
 1   result  174 non-null    object
dtypes: object(2)
memory usage: 4.1+ KB


## Transformando as bases em colunas no dataframe

### tamanho gelo - milhões de m²

In [7]:
# Extrair o dicionário dentro de 'arcticData'
if data_ice is not None and "arcticData" in data_ice:
    arctic_data = data_ice["arcticData"]["data"]  
    table_ice = pd.DataFrame.from_dict(arctic_data, orient="index").reset_index()
    table_ice.columns = ["date", "value", "anom", "month"] 
    print(table_ice)
    #manter apenas a colunas date e value que importam para o estudo
    table_ice = table_ice[["date", "value"]]
    print("Tabela de Gelo:\n", table_ice)
 
else:
    print("Erro na exibição dos dados vindos da API de Gelo")

       date  value  anom  month
0    197901  20.81  1.79  19.02
1    197902  19.32  1.37  17.95
2    197903  20.34  1.23  19.11
3    197904  22.94  1.67  21.27
4    197905  24.69  1.47  23.22
..      ...    ...   ...    ...
546  202407  22.00 -2.82  24.82
547  202408  21.55 -2.72  24.27
548  202409  21.44 -2.76  24.20
549  202410  22.49 -3.23  25.72
550  202411  23.30 -2.71  26.01

[551 rows x 4 columns]
Tabela de Gelo:
        date  value
0    197901  20.81
1    197902  19.32
2    197903  20.34
3    197904  22.94
4    197905  24.69
..      ...    ...
546  202407  22.00
547  202408  21.55
548  202409  21.44
549  202410  22.49
550  202411  23.30

[551 rows x 2 columns]


In [8]:
# Transformando o campo date em apenas year
table_ice['year'] = table_ice['date'].str[:4].astype(int)
table_ice = table_ice[["year", "value"]]

print("Média Anual da Extensão do Gelo:\n", table_ice)


Média Anual da Extensão do Gelo:
      year  value
0    1979  20.81
1    1979  19.32
2    1979  20.34
3    1979  22.94
4    1979  24.69
..    ...    ...
546  2024  22.00
547  2024  21.55
548  2024  21.44
549  2024  22.49
550  2024  23.30

[551 rows x 2 columns]


In [9]:
#calcular a média anual de 'value' - tentativa 1
media1 = (
    table_ice.groupby('year', as_index=False)['value'].mean()
    )
media1

,year,value
0,1979,24.005833
1,1980,23.554167
2,1981,23.533333
3,1982,24.062500
4,1983,23.697500
5,1984,23.348333
6,1985,23.585000
7,1986,23.268333
8,1987,-811.558333
9,1988,-811.220833


In [10]:
# Detectar e substituir outliers (negativos ou nulos)
def substituir_outliers_por_media(df, coluna_valor, coluna_ano):
    # Identificar índices dos valores negativos ou nulos
    outliers = df[df[coluna_valor] <= 0].index

    # Iterar sobre os outliers encontrados
    for idx in outliers:
        ano = df.loc[idx, coluna_ano]
        
        # Calcular a média excluindo o valor outlier
        media_sem_outlier = df[(df[coluna_ano] == ano) & (df.index != idx)][coluna_valor].mean()
        
        # Substituir o valor outlier pela média calculada
        df.loc[idx, coluna_valor] = media_sem_outlier

    return df

# Substituir valores negativos ou nulos na tabela
table_ice = substituir_outliers_por_media(table_ice, coluna_valor='value', coluna_ano='year')

# Calcular nova média anual após substituições
table_ice = table_ice.groupby('year', as_index=False)['value'].mean()
table_ice.rename(columns={'value': 'média_anual'}, inplace=True)
table_ice

,year,média_anual
0,1979,24.005833
1,1980,23.554167
2,1981,23.533333
3,1982,24.062500
4,1983,23.697500
5,1984,23.348333
6,1985,23.585000
7,1986,23.268333
8,1987,23.663636
9,1988,24.031818


### anomalia na temperatura de oceano

In [11]:
table_ocean = pd.DataFrame(data_ocean)
table_ocean

,error,result
1851,None,-0.05
1852,None,0.01
1853,None,0.05
1854,None,0.00
1855,None,-0.01
...,...,...
2020,None,0.75
2021,None,0.70
2022,None,0.64
2023,None,0.67


In [12]:
# Transformar a base de Oceanos em DF e remover a coluna error e adicionar year
if data_ocean is not None:
    table_ocean = pd.DataFrame(data_ocean)
    table_ocean = table_ocean.drop(columns=['error']).reset_index()
    table_ocean.columns = ['year', 'oceano - temp anomala']
    table_ocean['year'] = table_ocean['year'].astype(int)
    table_ocean['oceano - temp anomala'] = table_ocean['oceano - temp anomala'].astype(float)
    print("Global Ocean Temperature Anomalies:\n", table_ocean)
else:
    print("Erro transformação API de Oceano")

Global Ocean Temperature Anomalies:
      year  oceano - temp anomala
0    1851                  -0.05
1    1852                   0.01
2    1853                   0.05
3    1854                   0.00
4    1855                  -0.01
..    ...                    ...
169  2020                   0.75
170  2021                   0.70
171  2022                   0.64
172  2023                   0.67
173  2024                   0.94

[174 rows x 2 columns]


### anomalia temperatura global

In [13]:
table_temp = pd.DataFrame(data_temp)
table_temp

,error,result
0,None,"{'time': '1880.04', 'station': '-0.29', 'land'..."
1,None,"{'time': '1880.13', 'station': '-0.48', 'land'..."
2,None,"{'time': '1880.21', 'station': '-0.32', 'land'..."
3,None,"{'time': '1880.29', 'station': '-0.52', 'land'..."
4,None,"{'time': '1880.38', 'station': '-0.22', 'land'..."
...,...,...
1734,None,"{'time': '2024.54', 'station': '1.46', 'land':..."
1735,None,"{'time': '2024.63', 'station': '1.65', 'land':..."
1736,None,"{'time': '2024.71', 'station': '1.49', 'land':..."
1737,None,"{'time': '2024.79', 'station': '1.63', 'land':..."


In [14]:
# Transformar a base de Temperaturas com tolist()
if data_temp is not None and not data_temp.empty and "result" in data_temp:
    table_temp = pd.DataFrame(data_temp["result"].tolist()) 
    table_temp['year'] = table_temp['time'].astype(str).str[:4].astype(int)
    convert = ['station', 'land']
    table_temp[convert] = table_temp[convert].astype(float)
    
    # Agrupar por ano e calcular a média para 'station' e 'land'
    table_temp = (
        table_temp.groupby('year')[convert].mean().reset_index()
        .rename(columns={'station': 'temp_media_station', 'land': 'temp_media_land'})
    )
    print("Média anual de 'station' e 'land':\n", table_temp)
else:
    print("Erro transformação API Temperaturas")


Média anual de 'station' e 'land':
      year  temp_media_station  temp_media_land
0    1880           -0.323333        -0.172500
1    1881           -0.263333        -0.089167
2    1882           -0.370000        -0.107500
3    1883           -0.351667        -0.174167
4    1884           -0.494167        -0.283333
..    ...                 ...              ...
140  2020            1.310833         1.005833
141  2021            1.145000         0.844167
142  2022            1.160833         0.890000
143  2023            1.435000         1.170833
144  2024            1.552727         1.285455

[145 rows x 3 columns]


# Checagem

In [15]:
table_ice.info()
table_temp.info()
table_ocean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         46 non-null     int64  
 1   média_anual  46 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 868.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                145 non-null    int64  
 1   temp_media_station  145 non-null    float64
 2   temp_media_land     145 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 3.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   174 non-null    int64  
 1   oceano - te

In [16]:
table_ocean

,year,oceano - temp anomala
0,1851,-0.05
1,1852,0.01
2,1853,0.05
3,1854,0.00
4,1855,-0.01
...,...,...
169,2020,0.75
170,2021,0.70
171,2022,0.64
172,2023,0.67


In [17]:
table_ice

,year,média_anual
0,1979,24.005833
1,1980,23.554167
2,1981,23.533333
3,1982,24.062500
4,1983,23.697500
5,1984,23.348333
6,1985,23.585000
7,1986,23.268333
8,1987,23.663636
9,1988,24.031818


In [18]:
table_temp

,year,temp_media_station,temp_media_land
0,1880,-0.323333,-0.172500
1,1881,-0.263333,-0.089167
2,1882,-0.370000,-0.107500
3,1883,-0.351667,-0.174167
4,1884,-0.494167,-0.283333
...,...,...,...
140,2020,1.310833,1.005833
141,2021,1.145000,0.844167
142,2022,1.160833,0.890000
143,2023,1.435000,1.170833


# Unificar as tabelas

In [19]:
table_merged = pd.merge(table_ocean, table_ice, on='year', how='outer')
table_merged = pd.merge(table_merged, table_temp, on='year', how='outer')

print(table_merged.head())
print(table_merged.tail())

   year  oceano - temp anomala  média_anual  temp_media_station  \
0  1851                  -0.05          NaN                 NaN   
1  1852                   0.01          NaN                 NaN   
2  1853                   0.05          NaN                 NaN   
3  1854                   0.00          NaN                 NaN   
4  1855                  -0.01          NaN                 NaN   

   temp_media_land  
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              NaN  
     year  oceano - temp anomala  média_anual  temp_media_station  \
169  2020                   0.75    21.741667            1.310833   
170  2021                   0.70    22.103333            1.145000   
171  2022                   0.64    21.367500            1.160833   
172  2023                   0.67    20.303333            1.435000   
173  2024                   0.94    20.770909            1.552727   

     temp_media_land  
169         1.005833  
170       

# Salvando em banco de dados


In [20]:
import sqlite3
conn = sqlite3.connect('bd1.db')
table_merged.to_sql('table_merged', conn, if_exists='replace', index=False)


174

In [21]:
query = "SELECT * FROM table_merged"
df = pd.read_sql(query, conn)
print(df)
conn.close()

     year  oceano - temp anomala  média_anual  temp_media_station  \
0    1851                  -0.05          NaN                 NaN   
1    1852                   0.01          NaN                 NaN   
2    1853                   0.05          NaN                 NaN   
3    1854                   0.00          NaN                 NaN   
4    1855                  -0.01          NaN                 NaN   
..    ...                    ...          ...                 ...   
169  2020                   0.75    21.741667            1.310833   
170  2021                   0.70    22.103333            1.145000   
171  2022                   0.64    21.367500            1.160833   
172  2023                   0.67    20.303333            1.435000   
173  2024                   0.94    20.770909            1.552727   

     temp_media_land  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  
..               ...  
169       